In [1]:
from sklearn.utils import resample
import pandas as pd

# Load metadata
metadata = pd.read_csv("/Users/lucabernecker/PHD_UiT/Skin_lesions/HAM10000_metadata.csv")

# Check class distribution in the 'dx' column
class_counts = metadata['dx'].value_counts()

# Calculate the average class size
avg_class_size = int(class_counts.mean())

# Oversample minority classes to match the average class size
balanced_metadata = pd.concat(
    [
        
        resample(metadata[metadata['dx'] == label], 
                 replace=True,  # Enable replacement for oversampling
                 n_samples=avg_class_size, 
                 random_state=42)
        for label in class_counts.index
    ]
)

# Check new class distribution after oversampling
balanced_class_counts = balanced_metadata['dx'].value_counts()

# Output class distributions
class_counts, balanced_class_counts

(dx
 nv       6705
 mel      1113
 bkl      1099
 bcc       514
 akiec     327
 vasc      142
 df        115
 Name: count, dtype: int64,
 dx
 nv       1430
 mel      1430
 bkl      1430
 bcc      1430
 akiec    1430
 vasc     1430
 df       1430
 Name: count, dtype: int64)

In [2]:
categorical_columns = ['dx', 'sex', 'localization']
import numpy as np
print(balanced_metadata.head())
import pandas as pd
label_encoders = {}
from sklearn.preprocessing import LabelEncoder
for col in categorical_columns:
    le = LabelEncoder()
    balanced_metadata[col] = le.fit_transform(balanced_metadata[col])
    label_encoders[col] = le  # Store the encoder for inverse transformations if needed


print(balanced_metadata.head())
data = balanced_metadata
data['age'] = data['age'].fillna(data['age'].mean())
print(np.isnan(data["sex"]).sum())  # Check metadata
print(np.isnan(data["localization"]).sum())  # Check images
print(np.isnan(data["age"]).sum())  # Check labels
print(np.isnan(data["dx"]).sum())  # Check labels
print(len(np.unique(data['dx'])))

        lesion_id      image_id  dx    dx_type   age     sex localization
3835  HAM_0000474  ISIC_0030099  nv  follow_up  45.0  female         hand
8367  HAM_0000597  ISIC_0030654  nv      histo  35.0  female      abdomen
8203  HAM_0007585  ISIC_0032347  nv      histo  35.0  female         back
8168  HAM_0005902  ISIC_0027285  nv      histo  40.0  female         foot
6747  HAM_0004380  ISIC_0026251  nv      histo  30.0  female         face
        lesion_id      image_id  dx    dx_type   age  sex  localization
3835  HAM_0000474  ISIC_0030099   5  follow_up  45.0    0             8
8367  HAM_0000597  ISIC_0030654   5      histo  35.0    0             0
8203  HAM_0007585  ISIC_0032347   5      histo  35.0    0             2
8168  HAM_0005902  ISIC_0027285   5      histo  40.0    0             6
6747  HAM_0004380  ISIC_0026251   5      histo  30.0    0             5
0
0
0
0
7


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import Sequence, to_categorical
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import MobileNetV3Small, ResNet50, MobileNetV3Large

# Set the image directory path
image_dir = "/Users/lucabernecker/PHD_UiT/Skin_lesions/HAM10000_images_part_1"
data = balanced_metadata
# Image dimensions and batch size
IMG_SIZE = (450, 450,3)
BATCH_SIZE = 32

class_mapping = {label: idx for idx, label in enumerate(balanced_metadata['dx'].unique())}

class DataGenerator(Sequence):
    def __init__(self, dataframe, image_dir, batch_size, augment=False):
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.batch_size = batch_size
        self.indices = np.arange(len(dataframe))
        self.augment = augment  # Flag to enable/disable augmentation

    def __len__(self):
        return int(np.ceil(len(self.dataframe) / self.batch_size))
    
    def __getitem__(self, index):
        # Get batch indices
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch_data = self.dataframe.iloc[batch_indices]
        
        # Process inputs (e.g., images) and labels
        X_images = np.array([
            self.load_image(os.path.join(self.image_dir, f"{image_id}.jpg"))
            for image_id in batch_data['image_id']
        ], dtype=np.float32)  # Cast to float32 for consistency
        
        if self.augment:
            X_images = self.augment_images(X_images)  # Apply augmentation
        
        # Prepare labels
        y = batch_data['dx'].values
        y = to_categorical(y, num_classes=7)
        
        return X_images, y
    
    def load_image(self, image_path):
        img = tf.keras.preprocessing.image.load_img(image_path, target_size=(450, 450, 3))
        img = tf.keras.preprocessing.image.img_to_array(img)
        img = img / 255.0  # Normalize pixel values to [0, 1]
        return img

    def augment_images(self, images):
        augmented_images = []
        for img in images:
            # Apply random augmentations using tf.image
            img = tf.image.random_flip_left_right(img)  # Random horizontal flip
            img = tf.image.random_flip_up_down(img)  # Random vertical flip
            img = tf.image.random_brightness(img, max_delta=0.1)  # Random brightness adjustment
            img = tf.image.random_contrast(img, lower=0.8, upper=1.2)  # Random contrast adjustment
            img = tf.image.random_saturation(img, lower=0.8, upper=1.2)  # Random saturation adjustment
            img = tf.keras.preprocessing.image.random_shift(
                img,
                wrg=0.1,  # Shift up to 10% of the width
                hrg=0.1,  # Shift up to 10% of the height
                row_axis=1,  # Axis for height
                col_axis=2,  # Axis for width
                channel_axis=0,  # Axis for channels
                fill_mode='nearest',  # Fill empty space with nearest pixel
                cval=0.0,  # Not used here since fill_mode='nearest'
                interpolation_order=1  # Bilinear interpolation
            )
            img = tf.keras.preprocessing.image.random_shear(
                img,
                intensity=0.1,
                row_axis=1,
                col_axis=2,
                channel_axis=0,
                fill_mode='nearest',
                cval=0.0,
                interpolation_order=1
            )
            img = tf.keras.preprocessing.image.random_rotation(
                    img,
                    rg = 10,
                    row_axis=1,
                    col_axis=2,
                    channel_axis=0,
                    fill_mode='nearest',
                    cval=0.0,
                    interpolation_order=1
                )
            augmented_images.append(img)
        return np.array(augmented_images, dtype=np.float32)



# Split the balanced data into train, validation, and test sets
# Hyperparameters
train_df, val_df = train_test_split(data, test_size=0.2, random_state=42)

# Hyperparameters
batch_size = 16
input_shape = (450, 450, 3)  # For image data

# Create the training and validation generators
train_generator = DataGenerator(
    dataframe=train_df,
    image_dir=image_dir,
    batch_size=batch_size,
    augment=True  
)

val_generator = DataGenerator(
    dataframe=val_df,
    image_dir=image_dir,
    batch_size=batch_size,
    augment=False  
)

# Build a CNN model
model = ResNet50(
    input_shape=(450, 450, 3),  # Note: input_shape must include the channel dimension (RGB: 3 channels)
    include_top=True,
    weights=None,  # No pre-trained weights since you're training from scratch
    classes=len(balanced_metadata['dx'].unique()),  # Ensure correct number of output classes
    classifier_activation='softmax',  # Activation for multi-class classification
)
# Compile the model
model.compile(optimizer='SGD',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20, 
    steps_per_epoch=len(train_generator),
    validation_steps=len(val_generator),
)
 
# Evaluate on the test set
test_loss, test_accuracy = model.evaluate(test_gen)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

2025-01-10 10:47:16.275615: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-01-10 10:47:16.275635: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-01-10 10:47:16.275640: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2025-01-10 10:47:16.275653: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-01-10 10:47:16.275661: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Users/lucabernecker/anaconda3/envs/tf-macos/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call 

Epoch 1/20


2025-01-10 10:47:18.650233: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


501/501 ━━━━━━━━━━━━━━━━━━━━ 825s 2s/step - accuracy: 0.2851 - loss: 2.2899 - val_accuracy: 0.3437 - val_loss: 1.8068
Epoch 2/20
501/501 ━━━━━━━━━━━━━━━━━━━━ 768s 2s/step - accuracy: 0.4414 - loss: 1.4660 - val_accuracy: 0.3352 - val_loss: 2.5224
Epoch 3/20
501/501 ━━━━━━━━━━━━━━━━━━━━ 9487s 19s/step - accuracy: 0.5131 - loss: 1.3018 - val_accuracy: 0.3027 - val_loss: 2.3626
Epoch 4/20
125/501 ━━━━━━━━━━━━━━━━━━━━ 9:06 1s/step - accuracy: 0.5690 - loss: 1.1325 